In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
# Check whether google drive is mounted
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


In [ ]:
!pip install pyradiomics
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
import subprocess
from tqdm import tqdm
from radiomics import featureextractor
from scipy.ndimage import binary_opening
from skimage.measure import regionprops, label
import SimpleITK as sitk

# Path

In [ ]:
root_dir = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver/'
excel_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/classifier/'
save_dir = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/results/'
log_dir = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/log/'


# 把檔名放在excel檔內去讀取
test_data = pd.read_excel(excel_path + "all_data.xlsx", sheet_name='all')

ID = test_data['image'].values
phase = "PV"#"PV" #"Reg_A" #"Reg_pre"
img_pths = root_dir+ID[:]+"/"+phase+".nii.gz"

# Groung truth
mask_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[label]/'
lab_pths = mask_path+ID[:]+".nii.gz"
dst = os.path.join(save_dir, 'GT_all2.csv')
log_pth = os.path.join(log_dir, 'GT_all2.csv')

# 模型切割的label
# mask_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/'
# p = "PV"
# lab_pths = mask_path+ID[:]+"_"+p+".nii.gz"
# dst = os.path.join(save_dir, 'PV_all.csv')
# log_pth = os.path.join(log_dir, 'PV_all.csv')

# test_lab = test_data['label'].values
# lab_pths = mask_path+ID[:]+".nii.gz"

print(f'Images: {len(img_pths)}')
print(f'Masks: {len(lab_pths)}')

os.makedirs(save_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)



Images: 18
Masks: 18


In [ ]:
radiomics_setting_fpath = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/radiomics_setting.yaml'
extractor = featureextractor.RadiomicsFeatureExtractor(radiomics_setting_fpath)

INFO:radiomics.featureextractor:Loading parameter file /content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/radiomics_setting.yaml


In [ ]:
def save_result(dst, df=None, sub_df=None):
    # save result
    if os.path.exists(dst):
        df = pd.concat([df, sub_df])
        df.index = np.arange(1, len(df)+1)
        df.to_csv(dst, index=False)
        print(f'Save result to {dst}')
    else:
        sub_df.index = np.arange(1, len(sub_df)+1)
        sub_df.to_csv(dst, index=False)
        print(f'Save result to {dst}')

In [ ]:
def extract_radiomics_func(dst, log_pth, img, seg, pid, img_pth, seg_pth):
    err_pid = []
    err_log = []
    df = None
    global err_df

    if os.path.exists(dst):
        df = pd.read_csv(dst, dtype=str)
    if os.path.exists(log_pth):
        err_df = pd.read_csv(log_pth, dtype=str)
    # try:
    result = extractor.execute(img, seg)
    # collect result
    columns = []
    values = []
    columns.append('PatientID')
    values.append(pid)
    columns.append('ImageFile')
    values.append(img_pth)
    columns.append('LabelFile')
    values.append(seg_pth)

    for item in result:
        columns.append(item)
        values.append(result[item])
    sub_df = pd.DataFrame([values], columns=columns)
    print('Done')

    save_result(dst, df, sub_df)

    # except Exception as e:
    #     err_pid.append(pid)
    #     err_log.append(e)
    #     sub_err_df = pd.DataFrame({'PatientID': err_pid, 'Log': err_log})
        # save_result(log_pth, err_df, sub_err_df)

# Run

In [ ]:
!pip install antspyx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ants
from scipy import ndimage
from skimage import measure,color

for i, img_pth in enumerate(img_pths):
    pid = ID[i]
    seg_pth = lab_pths[i]
    print(lab_pths[i])
    if os.path.exists(seg_pth) and os.path.exists(img_pth):
      # Read files
      image = ants.image_read(img_pth)
      image = ants.resample_image(image,(1,1,1),0,0)
      ants.image_write(image, "/tmp/tmp_image.nii.gz")
      img = sitk.ReadImage("/tmp/tmp_image.nii.gz")

      img_label = ants.image_read(seg_pth)
      test_mask = ants.resample_image(img_label,(1,1,1),0,1)

      ## 只有在 label == 2 的地方會有值
      tumor_seg = np.zeros(test_mask.shape)
      tumor_seg[test_mask.numpy() == 2] = 1 #要對應yaml內的值
      tumor_seg = test_mask.new_image_like(tumor_seg)

      ## Closing: removes small holes
      ## https://antspy.readthedocs.io/en/latest/utils.html
      # label = ants.get_mask(tumor_seg, low_thresh=None, high_thresh=None, cleanup=2)

      ants.image_write(tumor_seg, "/tmp/tmp.nii.gz")
      image_label = nib.load("/tmp/tmp.nii.gz")
      label = image_label.get_fdata()

      '''****** <Connected-component labeling> ******'''
      area = 0
      labels = measure.label(label,connectivity=2)  ## CCL #8連通區域標記 #1代表4鄰接，2代表8鄰接
      properties = measure.regionprops(labels)
      tumor_size = []

      for prop in properties:
        if prop.area > area:
          area = prop.area
          x1, y1, z1, x2, y2, z2 = prop.bbox

      volume_Xmin, volume_Xmax, volume_Ymin, volume_Ymax,volume_Zmin, volume_Zmax = x1, x2, y1, y2, z1, z2
      label_ccl = label[volume_Xmin:volume_Xmax, volume_Ymin:volume_Ymax, volume_Zmin:volume_Zmax]

      labelCCL = nib.Nifti1Image(label_ccl, image_label.affine)
      nib.save(labelCCL, "/tmp/tmp2.nii.gz")

      seg = sitk.Cast(sitk.ReadImage("/tmp/tmp2.nii.gz"), sitk.sitkUInt8)
      seg.SetDirection(img.GetDirection())
      seg.SetOrigin(img.GetOrigin())
      seg.SetSpacing(img.GetSpacing())


      try:
        extract_radiomics_func(dst, log_pth, img, seg, pid, img_pth, seg_pth)
      except ValueError:
        continue
    else:
      continue
    #break

/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[label]/HCC_088.nii.gz


INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.imageoperations:New spacing equal to original image spacing, just resampling the mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.imageoperations:Computing LoG with sigm

Done
Save result to /content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/results/GT_all2.csv
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[label]/HCC_089.nii.gz


INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.imageoperations:New spacing equal to original image spacing, just resampling the mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.imageoperations:Computing LoG with sigm

Done
Save result to /content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/results/GT_all2.csv
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[label]/HCC_090.nii.gz


...

Done
Save result to /content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/results/GT_all2.csv
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[label]/HCC_104.nii.gz


INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.imageoperations:New spacing equal to original image spacing, just resampling the mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.imageoperations:Computing LoG with sigm

Done
Save result to /content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/results/GT_all2.csv
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[label]/HCC_105.nii.gz


INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.imageoperations:New spacing equal to original image spacing, just resampling the mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.imageoperations:Computing LoG with sigm

Done
Save result to /content/drive/MyDrive/Colab_Notebooks/liver_cancer/radiomics/results/GT_all2.csv
